# Save fitted spectra SSP

From fitted parameters it generate the Spectra into an hdf5 format (lile the Fors2-SL Templates)

- Author Sylvie Dagoret-Campagne
- Afflilation : IJCLab/IN2P3/CNRS
- Organisation : LSST-DESC
- creation date : 2023-11-27
- 
- conda environmment on my laptop conda_jaxcpu_dsps_py310
- 
- update : 2023-12-10 : View fitted spectra for run fit of november 27th 2023
- last update : 2024-01-06 : View fitted spectra for run fit of January 6th 2024
- last update : 2024-01-29 : View fitted spectra for run fit of January 29th 2024

In [ ]:
import h5py
import pandas as pd
import numpy as np
import os
import re
import pickle 
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.colors as colors
import matplotlib.cm as cmx
import collections
from collections import OrderedDict
import re
import matplotlib.gridspec as gridspec

In [ ]:
plt.rcParams["figure.figsize"] = (12,6)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'
plt.rcParams['legend.fontsize']=  16

## imports

### fitter jaxopt

In [ ]:
from fors2tostellarpopsynthesis.fitters.fitter_jaxopt import (SSP_DATA,mean_spectrum,mean_mags,mean_sfr,ssp_spectrum_fromparam)

In [ ]:
from dsps.cosmology import DEFAULT_COSMOLOGY, age_at_z

### parameters

In [ ]:
from fors2tostellarpopsynthesis.parameters import SSPParametersFit,paramslist_to_dict

### StarLight

In [ ]:
from fors2tostellarpopsynthesis.fors2starlightio import SLDataAcess, flux_norm

## Configuration

### Constants

In [ ]:
wl0 = 3646.6

### fitted params

In [ ]:
# current date
#date = "2024-01-06"
date = "2024-01-28"
# previous date
#date = "2023-11-27"
suptitle = f"SED-DSPS-fit ({date})"

In [ ]:
input_path = "fitssp_results"
input_file = f"fitssp_results_{date}.h5"

In [ ]:
fullname_input = os.path.join(input_path ,input_file) 

### Functions

In [ ]:
def bluefraction(x,y,wlcut=wl0):

    indexes_blue = np.where(x<wlcut)[0]
    indexes_red =np.where(x>wlcut)[0]

    integ_blue = np.trapz(y[indexes_blue],x[indexes_blue])
    integ_red = np.trapz(y[indexes_red],x[indexes_red])

    fraction_blue = integ_blue /(integ_blue+integ_red)
    fraction_red= integ_red /(integ_blue+integ_red)

    return fraction_blue

## Read Dataframe 

In [ ]:
df = pd.read_hdf(fullname_input)

In [ ]:
NF = len(df)

## Decode fitted parameters and plot the corresponding spectra

In [ ]:
names_fitparams = list(df.columns[8:])

In [ ]:
all_x = []
all_fnu_nodust = []
all_fnu_dust = []

all_fl_nodust = []
all_fl_dust = []


all_fractions = []


for idx in range(NF):

    row = df.iloc[idx]
    z_obs = row["zobs"]
    specname = row["fors2name"]
    list_params = row[names_fitparams].values
    dict_params = paramslist_to_dict(list_params,names_fitparams)
    x,fnu_nodust,fnu_dust = ssp_spectrum_fromparam(dict_params,z_obs)

    norm_y_nodust = flux_norm(x,fnu_nodust)
    norm_y_dust = flux_norm(x,fnu_dust)

    fnu_nodust /=  norm_y_nodust
    fnu_dust /=  norm_y_dust
 
    fract_blue = bluefraction(x,fnu_nodust)

    fl_nodust =  fnu_nodust/x**2
    fl_dust =  fnu_dust/x**2

    norm_y_nodust = flux_norm(x,fl_nodust)
    norm_y_dust = flux_norm(x,fl_dust)

    fl_nodust /=  norm_y_nodust
    fl_dust /=  norm_y_dust
    
    all_x.append(x)
    all_fnu_nodust.append(fnu_nodust) 
    all_fnu_dust.append(fnu_dust) 
    all_fractions.append(fract_blue)

    all_fl_nodust.append(fl_nodust) 
    all_fl_dust.append(fl_dust) 

    

all_fractions = np.array(all_fractions)


## Check the spectra by plotting

In [ ]:
bwr_map = plt.get_cmap('bwr')
reversed_map = bwr_map.reversed() 
cNorm = colors.Normalize(vmin=np.log10(all_fractions).min(), vmax=np.log10(all_fractions).max())
scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=reversed_map)
all_colors = scalarMap.to_rgba(np.log10(all_fractions), alpha=1)

In [ ]:
_, ax1 = plt.subplots(1, 1,figsize=(10,6))


__= ax1.set_yscale('log') 
__= ax1.set_xscale('log') 

for idx in range(NF):
    ax1.plot(all_x[idx],all_fnu_nodust[idx],'-',color=all_colors[idx])

   
__= ax1.set_xlim(100.,1e6)
__= ax1.set_ylim(1e-7,100)

ax1.set_xlabel("$\lambda (\\AA)$")
ax1.set_ylabel("$L_\\nu(\lambda)$ relative flux")
ax1.set_title("$f_\\nu(\lambda)$ Fitted spectra")
ax1.grid()
ax1.axvline(wl0,color="k")

plt.suptitle(suptitle)

In [ ]:
_, ax1 = plt.subplots(1, 1,figsize=(10,6))
__= ax1.set_yscale('log') 
__= ax1.set_xscale('log') 

for idx in range(NF):
    ax1.plot(all_x[idx],all_fl_nodust[idx],color=all_colors[idx])
__= ax1.set_xlim(100.,1e5)
__= ax1.set_ylim(1e-5,100)

ax1.axvline(wl0,color="k")
ax1.set_xlabel("$\lambda (\\AA)$")
ax1.set_ylabel("$L_\\lambda(\lambda)$ relative flux")
ax1.set_title("$f_\\lambda(\lambda)$ Fitted spectra")
ax1.grid()
plt.suptitle(suptitle)


## Create the HDF5  Spectrum structure and save the corresponding file

In [ ]:
output_file_h5  = f"FittedSSPspectra_{date}.hdf5"
hf_outfile =  h5py.File(output_file_h5, 'w') 

In [ ]:
list_params_names = list(df.columns)
NP = len(list_params_names)

In [ ]:
for idx in range(NF):
    row = df.iloc[idx]
    z_obs = row["zobs"]
    specname = row["fors2name"]
    list_params_values = row.values
   
    # create the group
    h5group = hf_outfile.create_group(specname)

    # loop on parameters
    for ip in range(NP):
        attr_name = list_params_names[ip]
        attr_val = list_params_values[ip]
        h5group.attrs[attr_name] = attr_val 
        
    d = h5group.create_dataset("wl",data=all_x[idx],compression="gzip", compression_opts=9)
    d = h5group.create_dataset("fl",data=all_fl_nodust[idx],compression="gzip", compression_opts=9)
    d = h5group.create_dataset("fl_dust",data=all_fl_dust[idx],compression="gzip", compression_opts=9)
    d = h5group.create_dataset("fnu",data=all_fnu_nodust[idx],compression="gzip", compression_opts=9)
    d = h5group.create_dataset("fnu_dust",data=all_fnu_dust[idx],compression="gzip", compression_opts=9)

In [ ]:
#hf_outfile.keys()

In [ ]:
hf_outfile.close() 

## Read again the written file to check

In [ ]:
input_file_h5  = output_file_h5 

In [ ]:
if os.path.isfile(input_file_h5):
    hf = h5py.File(input_file_h5, 'r')
    list_of_groupkeys = list(hf.keys())
    # pick one key
    key_sel =  list_of_groupkeys[0]
    # pick one group
    group = hf.get(key_sel)
    #pickup all attribute names
    list_of_subgroup_keys = []
    for k in group.attrs.keys():
        list_of_subgroup_keys.append(k)
else:
    hf = None
    list_of_groupkeys = []
    list_of_subgroup_keys = []

In [ ]:
print(list_of_groupkeys[:10])

In [ ]:
list_of_subgroup_keys

In [ ]:
groupname = list_of_groupkeys[0]

In [ ]:
attr_dict = OrderedDict()
if groupname in list_of_groupkeys:
    group = hf.get(groupname)
    for nameval in list_of_subgroup_keys:
        attr_dict[nameval] = group.attrs[nameval]

In [ ]:
attr_dict

In [ ]:
group = hf.get(groupname)
wl = np.array(group.get("wl"))
fld = np.array(group.get("fl_dust"))
fl = np.array(group.get("fl"))

In [ ]:
_, ax = plt.subplots(1, 1,figsize=(10,4))
__= ax.set_yscale('log') 
__= ax.set_xscale('log') 
ax.plot(wl,fl,'b-') 
ax.plot(wl,fld,'r:')
__= ax.set_xlim(100.,1e5)
__= ax.set_ylim(1e-5,100)
